In [1]:
# from skimage import transform # io, 
# import PIL
# import math
# from glob import glob

# import matplotlib.pyplot as plt
# from torch.utils.data import DataLoader
# from torchvision import transforms, utils, datasets, models
# import torch.optim as optim
# from torch.optim import lr_scheduler
# import torchvision
# import pickle

# # Ignore warnings
# import warnings
# warnings.filterwarnings("ignore")

# %matplotlib inline
# plt.ion()   # interactive mode


# from visualize import show_2Dbatch, show_4channel_batch, show_3Dbatch

In [2]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

# can revome the lines that need these
import os
import pickle

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from datasets import GeneralDataset
import Transforms as myTransforms

%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
# choose task: either 'idh' or '1p19q'
task = 'idh'

# choose mtl: True or False
mtl = False

# choose MR input
## details ('crop3Dslice', 'modality3D'
dataformat = 'modality3D'

# modality (either None, 'T1ce', 'FLAIR', 'T2', 'T1', 'T1ce-T1')
modality = None

# metadata for all brats (including tcia) data
glioma_metadata_df = pd.read_csv('../data/glioma_metadata.csv', index_col=0)
glioma_metadata_df.loc[['Brats18_TCIA09_462_1', 'Brats18_TCIA10_236_1'], 'idh'] = 1

if task == 'idh':
    classes = ['wildtype', 'mutant']
    val_df = glioma_metadata_df.loc[(glioma_metadata_df['phase'] == 'val') & (glioma_metadata_df['idh'].isin([0,1]))]
    if mtl:
        train_df = glioma_metadata_df.loc[glioma_metadata_df['phase'].isin(['train', 'unlabeled train'])]
    else:
        train_df = glioma_metadata_df.loc[(glioma_metadata_df['phase'] == 'train') & (glioma_metadata_df['idh'].isin([0,1]))]
        
elif task == '1p19q':
    classes = ['non-codel', 'oligo']
    val_df = glioma_metadata_df.loc[glioma_metadata_df['phase'] == 'val']

# map between brats dataset and tcia data (tcia data is avalible for a subset of the brats patients)
# brats2tcia_df = pd.read_csv('../data/brats2tcia_df_542x1.csv', index_col=0)
brats2tcia_df =glioma_metadata_df['tciaID']


# these are labeled files (they were paths in old dataloader) but they are dataframes
csv_files = {'train':train_df, 'val':val_df, 'data':glioma_metadata_df}


genomic_csv_files = {'train':'../data/MGL/MGL_235x50.csv', 
                     'val':'../data/MGL/MGL_235x50.csv',
                     'data':'../data/MGL/MGL_235x50.csv'}


image_dir = '../data/all_brats_scans/'
cluster_column = task

train_batch_size = 4
val_batch_size = 4
data_batch_size = 1



# resize_shape = (16, 16, 16)
channels = 4
interpolation = 1

shuffle = True
shuffle_data = False

resize_shape = (64, 64, 64)
print('Train size', len(train_df))

Train size 112


# 3D cropped

In [5]:
train_transformations = myTransforms.Compose([
        myTransforms.MinMaxNormalize(),
        myTransforms.ScaleToFixed((channels, resize_shape[0],resize_shape[1],resize_shape[2]), 
                                  interpolation=interpolation, 
                                  channels=channels),
        myTransforms.ZeroSprinkle(prob_zero=0.2, prob_true=0.8),
        myTransforms.ZeroChannel(prob_zero=0.5),
        myTransforms.RandomFlip(),
        myTransforms.ToTensor(),
    ])

seg_transformations = myTransforms.Compose([
    myTransforms.ScaleToFixed((1, resize_shape[0],resize_shape[1],resize_shape[2]), 
                                  interpolation=0, 
                                  channels=1),
        myTransforms.ToTensor(),
    ])

seg_probs_transformations = myTransforms.Compose([
        myTransforms.ScaleToFixed((channels, resize_shape[0],resize_shape[1],resize_shape[2]), 
                                  interpolation=interpolation, 
                                  channels=channels),
        myTransforms.ToTensor(),
    ])

val_transformations = myTransforms.Compose([
        myTransforms.MinMaxNormalize(),
        myTransforms.ScaleToFixed((channels, resize_shape[0],resize_shape[1],resize_shape[2]), 
                                  interpolation=interpolation, 
                                  channels=channels),
#         myTransforms.RandomFlip(),
        myTransforms.ToTensor(),
    ])

data_transforms = {'train': train_transformations,
                   'val':   val_transformations,
                   'data':   val_transformations,
                   'seg': seg_transformations,
                   'seg_probs':seg_probs_transformations
                  }



transformed_dataset_train = GeneralDataset(csv_file=csv_files['train'],
                                           root_dir=image_dir,
                                           genomic_csv_file = genomic_csv_files['train'],
                                           transform=data_transforms['train'],
                                           seg_transform=data_transforms['seg'],
                                           seg_probs_transform=data_transforms['seg_probs'],
                                           label='cluster',
                                           classes=classes,
                                           dataformat=dataformat,
                                           returndims=resize_shape,
                                           return_max_slice=False,
                                           axial_only=True,
                                           cluster_column=cluster_column,
                                           brats2tcia_df=brats2tcia_df,
                                           modality=None)

transformed_dataset_val = GeneralDataset(csv_file=csv_files['val'],
                                         root_dir=image_dir,
                                         genomic_csv_file = genomic_csv_files['val'],
                                         transform=data_transforms['val'],
                                         seg_transform=data_transforms['seg'],
                                         seg_probs_transform=data_transforms['seg_probs'],
                                         label='cluster',
                                         classes=classes,
                                         dataformat=dataformat,
                                         returndims=resize_shape,
                                         cluster_column=cluster_column,
                                         brats2tcia_df=brats2tcia_df,
                                         modality=None)


transformed_dataset_data = GeneralDataset(csv_file=csv_files['data'],
                                         root_dir=image_dir,
                                          genomic_csv_file = genomic_csv_files['data'],
                                         transform=data_transforms['data'],
                                          seg_transform=data_transforms['seg'],
                                          seg_probs_transform=data_transforms['seg_probs'],
                                         label='cluster',
                                         classes=classes,
                                         dataformat=dataformat,
                                         returndims=resize_shape,
                                         cluster_column=cluster_column,
                                         brats2tcia_df=brats2tcia_df,
                                         modality=None)

image_datasets = {'train':transformed_dataset_train, 
                  'val':transformed_dataset_val,
                  'data':transformed_dataset_data}


dataloader_train = DataLoader(image_datasets['train'], batch_size=train_batch_size, shuffle=shuffle, num_workers=4)
dataloader_val = DataLoader(image_datasets['val'], batch_size=val_batch_size, shuffle=shuffle, num_workers=4)
dataloader_data = DataLoader(image_datasets['data'], batch_size=data_batch_size, shuffle=shuffle_data, num_workers=4)

dataloaders = {'train':dataloader_train, 'val':dataloader_val, 'data':dataloader_data}

dataset_sizes = {'train':len(image_datasets['train']), 
                 'val':len(image_datasets['val']), 
                 'data':len(image_datasets['data'])}

class_names = image_datasets['train'].classes
class_names

TypeError: __init__() got an unexpected keyword argument 'seg_probs_transform'

In [ ]:
# subtype_dict = {0:'wildtype', 1:'val'}
# for i, data in enumerate(dataloaders['train']):
#     (inputs, seg_image, genomic_data, seg_probs), labels,(OS, event), bratsID = data
#     inputs, labels = inputs.to(device), labels.to(device)
#     print(np.unique(seg_probs))
    
# #     seg_image2 = seg_probs[0].max(1)[1].data.byte().cpu().numpy()
    
#     print('**', inputs.shape)
#     break

In [5]:
from torch.utils.tensorboard import SummaryWriter
img_dims = str(resize_shape[0]) + 'x' + str(resize_shape[1]) + 'x' + str(resize_shape[2])
model_outfile_dir = '3D_' + task + '_croppedraw_' + img_dims +'_bs-' + str(train_batch_size) + '_newdataloader_only-gt-idh'
print('tensorboad:', model_outfile_dir)
writer = SummaryWriter('runs1/'+model_outfile_dir)

tensorboad: 3D_idh_croppedraw_64x64x64_bs-4_newdataloader_only-gt-idh


In [ ]:
num_classes = 2
best_model_loc = '../pretrained/espnet_3d_brats.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cluster_df = csv_files['train'][task]
_, cnts = np.unique(cluster_df, return_counts=True)
loss_weights = (np.ones(num_classes)/cnts)*np.max(cnts)
loss_weights = torch.FloatTensor(loss_weights).to(device)
criterion = nn.CrossEntropyLoss(weight=loss_weights)
print('loss weights:', loss_weights)


from train import train
from models.Models import SegModel
best_auc_list, best_acc_list, best_auc_acc_list = [], [], []
epochs = 50
iterations = 10
for i in range(iterations):
    print('Iteration', i)

    best_model_loc = '../pretrained/espnet_3d_brats.pth'
    esp_model = SegModel(best_model_loc=best_model_loc, inp_res = resize_shape, num_classes=num_classes)
    esp_model = esp_model.to(device=device)

    optimizer_esp = optim.Adam(esp_model.parameters(), lr=0.0005) # change to adam
    exp_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_esp, 
                                                         mode='min', 
                                                         factor=0.1, 
                                                         patience=10, # number of epochs with no change 
                                                         verbose=True, 
                                                         threshold=0.0001, 
                                                         threshold_mode='rel', 
                                                         cooldown=0, 
                                                         min_lr=0, 
                                                         eps=1e-08)


    model, best_wts, best_auc, best_acc, best_auc_acc = train(model=esp_model, 
                       dataloaders=dataloaders,
                       data_transforms=data_transforms,
                       criterion = criterion, 
                       optimizer=optimizer_esp, 
                       scheduler=exp_scheduler,
                       writer=writer,
                       num_epochs=epochs, 
                       verbose=False, 
                       device=device,
                       dataset_sizes=dataset_sizes,
                       channels=1,
                       resize_shape=resize_shape,
                       classes=class_names,
                       weight_outfile_prefix=model_outfile_dir)
    del esp_model
    del model
    
    best_auc_list.append(best_auc)
    best_acc_list.append(best_acc)
    best_auc_acc_list.append(best_auc_acc)
    
    
    results_outfile_dir = model_outfile_dir + '_epochs-' + str(epochs) +'_iterations-' + str(iterations)
            
    if not os.path.exists('../model_weights/results/'):
        os.makedirs('../model_weights/results/')
            
    with open('../model_weights/results/auc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
        pickle.dump(best_auc_list, fp)

    with open('../model_weights/results/acc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
        pickle.dump(best_acc_list, fp)
    
    with open('../model_weights/results/avg_auc_acc_' + results_outfile_dir + '.txt', "wb") as fp:   #Pickling
        pickle.dump(best_auc_acc_list, fp)

loss weights: tensor([1.0364, 1.0000], device='cuda:0')
Iteration 0


  0%|          | 0/50 [00:00<?, ?it/s]

training . . . 


/app/software/scikit-learn/0.23.1-foss-2020a-Python-3.8.2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/app/software/scikit-learn/0.23.1-foss-2020a-Python-3.8.2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


New Best ACC:	 0.5 	in epoch 0
New Best AUC-acc aveage:	 0.6221064814814815 	in epoch 0


...

 24%|██▍       | 12/50 [05:32<17:33, 27.72s/it]

New Best AUC-acc aveage:	 0.6493055555555556 	in epoch 11
New Best ACC:	 0.6319444444444444 	in epoch 12
New Best AUC:	 0.7002314814814815 	in epoch 12


 26%|██▌       | 13/50 [06:01<17:13, 27.92s/it]

New Best AUC-acc aveage:	 0.666087962962963 	in epoch 12


 32%|███▏      | 16/50 [07:23<15:40, 27.66s/it]

In [ ]:
1

---
Two bad epochs (should redo) (March 8th)

50 epoch; 10 loops

3D_idh_croppedraw_64x64x64_bs-4_newdataloader

    auc std: 0.08738088661629888
    auc mean: 0.8121527777777778
    
    acc std: 0.08901614275582172
    acc mean: 0.7819444444444444
---
tensorboad: 3D_idh_croppedraw_64x64x64_bs-4_newdataloader_only-gt-idh (wrong optimizer!!)

    auc std: 0.02907397270859894
    auc mean: 0.871875
    
    acc std: 0.046718130289025175
    acc mean: 0.8065972222222222

In [ ]:
np.mean(best_acc_list)

In [ ]:
np.mean(best_auc_list)

In [ ]:
np.std(best_acc_list)

In [ ]:
temp = [ 0.8217592592592593,
 0.8668981481481481,
 0.8796296296296297,
 0.8263888888888888,
 0.8518518518518519,
 0.861111111111111,
 0.8553240740740741,
 0.8726851851851852]

In [ ]:
np.mean(temp)